In [1]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
from tensorflow.keras import optimizers
import tensorflow

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

170508288/170498071 [==============================] - 13s 0us/step


In [3]:
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [4]:
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [5]:
baseMapNum = 32
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                        

In [6]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [8]:
#training
batch_size = 64
epochs=5
opt_rms = tensorflow.keras.optimizers.RMSprop(learning_rate=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75.h5')

Epoch 1/15
781/781 [==============================] - 28s 34ms/step - loss: 0.6607 - accuracy: 0.8212 - val_loss: 0.6350 - val_accuracy: 0.8343
Epoch 2/15
781/781 [==============================] - 27s 35ms/step - loss: 0.6625 - accuracy: 0.8195 - val_loss: 0.5976 - val_accuracy: 0.8456
Epoch 3/15
781/781 [==============================] - 26s 34ms/step - loss: 0.6570 - accuracy: 0.8226 - val_loss: 0.6804 - val_accuracy: 0.8276
Epoch 4/15
781/781 [==============================] - 27s 34ms/step - loss: 0.6554 - accuracy: 0.8224 - val_loss: 0.6107 - val_accuracy: 0.8459
Epoch 5/15
781/781 [==============================] - 27s 34ms/step - loss: 0.6520 - accuracy: 0.8255 - val_loss: 0.6858 - val_accuracy: 0.8303
Epoch 6/15
781/781 [==============================] - 26s 34ms/step - loss: 0.6476 - accuracy: 0.8271 - val_loss: 0.6511 - val_accuracy: 0.8342
Epoch 7/15
781/781 [==============================] - 28s 35ms/step - loss: 0.6456 - accuracy: 0.8283 - val_loss: 0.6179 - val_accuracy:

In [11]:
epochs=5

opt_rms = tensorflow.keras.optimizers.RMSprop(learning_rate=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100.h5')


Epoch 1/15
781/781 [==============================] - 28s 34ms/step - loss: 0.5245 - accuracy: 0.8645 - val_loss: 0.5140 - val_accuracy: 0.8719
Epoch 2/15
781/781 [==============================] - 27s 34ms/step - loss: 0.5191 - accuracy: 0.8642 - val_loss: 0.5811 - val_accuracy: 0.8545
Epoch 3/15
781/781 [==============================] - 27s 34ms/step - loss: 0.5159 - accuracy: 0.8667 - val_loss: 0.5548 - val_accuracy: 0.8615
Epoch 4/15
781/781 [==============================] - 27s 35ms/step - loss: 0.5130 - accuracy: 0.8651 - val_loss: 0.5562 - val_accuracy: 0.8592
Epoch 5/15
781/781 [==============================] - 26s 34ms/step - loss: 0.5129 - accuracy: 0.8648 - val_loss: 0.5331 - val_accuracy: 0.8687
Epoch 6/15
781/781 [==============================] - 26s 34ms/step - loss: 0.5127 - accuracy: 0.8658 - val_loss: 0.5617 - val_accuracy: 0.8587
Epoch 7/15
781/781 [==============================] - 27s 34ms/step - loss: 0.5096 - accuracy: 0.8681 - val_loss: 0.5153 - val_accuracy:

In [12]:
epochs=5

opt_rms = tensorflow.keras.optimizers.RMSprop(learning_rate=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125.h5')


Epoch 1/15
781/781 [==============================] - 28s 34ms/step - loss: 0.4719 - accuracy: 0.8794 - val_loss: 0.5149 - val_accuracy: 0.8715
Epoch 2/15
781/781 [==============================] - 27s 34ms/step - loss: 0.4618 - accuracy: 0.8792 - val_loss: 0.5121 - val_accuracy: 0.8741
Epoch 3/15
781/781 [==============================] - 27s 34ms/step - loss: 0.4551 - accuracy: 0.8818 - val_loss: 0.5226 - val_accuracy: 0.8736
Epoch 4/15
781/781 [==============================] - 26s 34ms/step - loss: 0.4549 - accuracy: 0.8823 - val_loss: 0.4925 - val_accuracy: 0.8786
Epoch 5/15
781/781 [==============================] - 26s 34ms/step - loss: 0.4527 - accuracy: 0.8822 - val_loss: 0.5150 - val_accuracy: 0.8737
Epoch 6/15
781/781 [==============================] - 26s 34ms/step - loss: 0.4491 - accuracy: 0.8834 - val_loss: 0.4784 - val_accuracy: 0.8807
Epoch 7/15
781/781 [==============================] - 26s 34ms/step - loss: 0.4486 - accuracy: 0.8818 - val_loss: 0.4938 - val_accuracy:

In [13]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

79/79 [==============================] - 1s 6ms/step - loss: 0.4954 - accuracy: 0.8747

Test result: 87.470 loss: 0.495
